In [3]:
from Bio import SeqIO
import pandas as pd
protein_seqs = {}
sequence_path = "/content/drive/MyDrive/project_1/sequences.fasta"
for record in SeqIO.parse(sequence_path, "fasta"):
  protein_seqs[record.id] = record.seq
print(len(protein_seqs))

8693


In [4]:
import transformers
from transformers import EsmModel, EsmConfig, AutoTokenizer, AutoModel
import torch
model_name = "facebook/esm2_t6_8M_UR50D"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/31.4M [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
embeddings = {}
for item in protein_seqs:
  inputs = tokenizer(item, return_tensors="pt")
  with torch.no_grad():
    outputs = model(**inputs)
  token_embeddings = outputs.last_hidden_state  # Shape: (1, sequence_length + 2, hidden_size)
  attention_mask = inputs['attention_mask']
  embeddings[item] = token_embeddings[0, 1:-1, :]


In [8]:
import pickle
#save embeddings to disk to save progress in case of crash
print(len(embeddings))
with open('embeddings.pickle', 'wb') as file:
    pickle.dump(embeddings, file)

8693


In [16]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [6]:
training_path = "/content/drive/MyDrive/project_1/train.tsv"
training = pd.read_csv(training_path, sep="\t")
print(training)

                   id secondary_structure
0          3KVH_LYS_6                   .
1          3KVH_GLN_7                   E
2          3KVH_ILE_8                   E
3          3KVH_SER_9                   .
4         3KVH_ARG_10                   H
...               ...                 ...
2708230  3HR6_ILE_430                   E
2708231  3HR6_ASP_431                   E
2708232  3HR6_ASN_432                   E
2708233  3HR6_LYS_433                   B
2708234  3HR6_LYS_434                   .

[2708235 rows x 2 columns]


In [15]:
import pickle
pickle_path = "/content/drive/MyDrive/project_1/embeddings.pickle"
with open(pickle_path, 'rb') as file:
  embeddings = pickle.load(file)
#print(embeddings['3KVH'].shape)

In [14]:
df = pd.DataFrame.from_dict(embeddings, orient='index')
column_name = 'embedding'
columns = [f'{column_name}_{i+1}' for i in range(len(df.iloc[0]))]
df.columns = columns
df.index.name = 'index'
df = df.reset_index()
df = df.dropna()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8693,) + inhomogeneous part.

In [17]:
for col in df.columns[2:]:
  col = LabelEncoder().fit_transform(df['island'])

NameError: name 'df' is not defined

In [ ]:
X = [col for col in df.columns[2:]]
y = df['position']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))